In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [24]:
from astroquery.gaia import Gaia
table = Gaia.load_table('gaiadr2.gaia_source')

Retrieving table 'gaiadr2.gaia_source'
Parsing table 'gaiadr2.gaia_source'...
Done.


In [2]:
from astropy.io import fits

In [22]:
data_file = 'data/GaiaDR2_vrad_vel.fits'
data_hdu = fits.open(data_file)
data_table = data_hdu[1].data
data_header = data_hdu[1].header
source_ids = data_table['source_id']
ras = data_table['ra']
decs = data_table['dec']

In [23]:
import astropy.units as u
from astropy.coordinates import SkyCoord

coord = SkyCoord(ra=ras[0], dec=[0], unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(1, u.arcsecond)
height = u.Quantity(1, u.arcsecond)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)

Launched query: 'SELECT DISTANCE(POINT('ICRS',ra,dec),                 POINT('ICRS',[45.13215011],[-2.13445257e-06])) AS dist, *                 FROM gaiadr1.gaia_source WHERE CONTAINS(                POINT('ICRS',ra,dec),                BOX('ICRS',[45.13215011],[-2.13445257e-06], 0.0002777777777777778, 0.0002777777777777778))=1                 ORDER BY dist ASC'
Retrieving async. results...
500


Exception: 500

In [ ]:
foo = Gaia.